In [1]:
from convokit import Corpus, download

In [2]:
corpus = Corpus(filename=download('subreddit-Cornell'))

Dataset already exists at /Users/calebchiam/.convokit/downloads/subreddit-Cornell


In [3]:
corpus.print_summary_stats()

Number of Speakers: 7568
Number of Utterances: 74467
Number of Conversations: 10744


## Bag-of-words prediction for utterances having positive scores

In [4]:
from convokit import BoWClassifier, BoWTransformer

In [5]:
bow_transformer = BoWTransformer(obj_type="utterance")

Initializing default unigram CountVectorizer...


In [6]:
print(next(corpus.iter_utterances()))

Utterance('id': 'nyx4d', 'root': nyx4d, 'reply-to': None, 'speaker': Speaker('id': reddmau5, 'meta': {'num_posts': 3, 'num_comments': 5}), 'timestamp': 1325452698, 'text': "I was just reading about the Princeton Mic-Check and it's getting [national press](http://www.bloomberg.com/news/2011-12-29/princeton-brews-trouble-for-us-1-percenters-commentary-by-michael-lewis.html).\n\nI want to get a sense of what people felt like around campus. Anything interesting happen? Anything interesting coming up?", 'meta': {'score': 2, 'top_level_comment': None, 'retrieved_on': -1, 'gilded': -1, 'gildings': None, 'subreddit': 'Cornell', 'stickied': False, 'permalink': '/r/Cornell/comments/nyx4d/so_i_was_away_this_past_semester_whats_going_on/', 'author_flair_text': ''})


In [7]:
bow_transformer.fit_transform(corpus)

In [8]:
print(next(corpus.iter_utterances()))

Utterance('id': 'nyx4d', 'root': nyx4d, 'reply-to': None, 'speaker': Speaker('id': reddmau5, 'meta': {'num_posts': 3, 'num_comments': 5}), 'timestamp': 1325452698, 'text': "I was just reading about the Princeton Mic-Check and it's getting [national press](http://www.bloomberg.com/news/2011-12-29/princeton-brews-trouble-for-us-1-percenters-commentary-by-michael-lewis.html).\n\nI want to get a sense of what people felt like around campus. Anything interesting happen? Anything interesting coming up?", 'meta': {'score': 2, 'top_level_comment': None, 'retrieved_on': -1, 'gilded': -1, 'gildings': None, 'subreddit': 'Cornell', 'stickied': False, 'permalink': '/r/Cornell/comments/nyx4d/so_i_was_away_this_past_semester_whats_going_on/', 'author_flair_text': '', 'bow_vector': <1x9340 sparse matrix of type '<class 'numpy.int64'>'
	with 42 stored elements in Compressed Sparse Row format>})


In [9]:
bow_classifier = BoWClassifier(obj_type="utterance", labeller=lambda utt: utt.meta['score'] > 0)

Initializing default classification model (standard scaled logistic regression)


In [10]:
bow_classifier.fit_transform(corpus)

In [11]:
next(corpus.iter_utterances()).meta

{'score': 2,
 'top_level_comment': None,
 'retrieved_on': -1,
 'gilded': -1,
 'gildings': None,
 'subreddit': 'Cornell',
 'stickied': False,
 'permalink': '/r/Cornell/comments/nyx4d/so_i_was_away_this_past_semester_whats_going_on/',
 'author_flair_text': '',
 'bow_vector': <1x9340 sparse matrix of type '<class 'numpy.int64'>'
 	with 42 stored elements in Compressed Sparse Row format>,
 'prediction': True,
 'pred_score': 0.9999998491105527}

In [12]:
next(corpus.iter_utterances()).meta['bow_vector'].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [13]:
bow_classifier.summarize(corpus).head()

,prediction,pred_score
id,,
dhhm9sa,True,1.0
dw553ml,True,1.0
dvzmhdx,True,1.0
dvzpp79,True,1.0
dw0imao,True,1.0


In [14]:
corpus.get_utterance('15enm8').text

'One, just to get this out of the way: I\'m only a sophomore in high school. In spite of this, my high school is one of the top public schools in New Jersey (and to put it bluntly it\'s a very affluent area... although I\'m not necessarily affluent like my classmates). The point of telling you guys that is kids start talking about all these amazing schools they want to go to in like eighth grade, so I know quite a bit about colleges. As stated in the title, I really want to go to Cornell, and I just was hoping that some of you guys and girls on here would be awesome enough to give out some SAT scores, ACT scores (if you took them), and extra curricular activities you guys got/did? My unweighted GPA is a 3.8 (weighted is a 4.2), and my first PSAT was an overall 1900, and from taking that I (not to sound cocky here) *know* that I\'m going to get that score up a *lot*. I\'m in all the highest level classes I can be in, and I\'m looking to take multiple AP courses next year (junior). Do yo

In [15]:
bow_classifier.get_coefs(feature_names=bow_transformer.get_vocabulary()).head()

,coef
feat_name,
hotels,1.270001
hbhs,1.115690
engine,1.109702
involves,1.081836
lincoln,1.071464


In [16]:
bow_classifier.get_coefs(feature_names=bow_transformer.get_vocabulary()).tail()

,coef
feat_name,
mahogany,-0.667785
ignoreme,-0.722992
hilton,-0.742234
binary,-0.764383
creation,-0.784593


In [17]:
y_true, y_pred = bow_classifier.get_y_true_pred(corpus)

In [18]:
bow_classifier.base_accuracy(corpus)

0.9279546644822538

In [19]:
bow_classifier.accuracy(corpus)

0.9491452589737737

In [20]:
print(bow_classifier.classification_report(corpus))

              precision    recall  f1-score   support

       False       0.88      0.34      0.49      5365
        True       0.95      1.00      0.97     69102

    accuracy                           0.95     74467
   macro avg       0.91      0.67      0.73     74467
weighted avg       0.95      0.95      0.94     74467



## Bag-of-words prediction for comment thread doubling in length versus staying the same length based on first 5 utterances

In [21]:
top_level_comment_ids = [utt.id for utt in corpus.iter_utterances() if utt.id == utt.meta['top_level_comment']]

In [22]:
corpus.print_summary_stats()

Number of Speakers: 7568
Number of Utterances: 74467
Number of Conversations: 10744


In [23]:
len(top_level_comment_ids)

32893

In [24]:
threads_corpus = corpus.reindex_conversations(new_convo_roots=top_level_comment_ids)


['c3od15i', 'c3oyf4d', 'c3p8bze', 'c3p1rn8', 'c3ocsyl']


In [25]:
threads_corpus.print_summary_stats()

Number of Speakers: 6160
Number of Utterances: 63697
Number of Conversations: 32888


In [26]:
for thread in threads_corpus.iter_conversations():
    thread_len = len(list(thread.iter_utterances()))
    if thread_len == 5:
        thread.meta['thread_doubles'] = False
    elif thread_len >= 10:
        thread.meta['thread_doubles'] = True
    else:
        thread.meta['thread_doubles'] = None

In [27]:
bow_transformer2 = BoWTransformer(obj_type="conversation", 
                    text_func=lambda convo: " ".join([utt.text for utt in convo.get_chronological_utterance_list()][:5]),
                   )

Initializing default unigram CountVectorizer...


In [28]:
bow_transformer2.fit_transform(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [29]:
bow_classifier2 = BoWClassifier(obj_type="conversation", 
                    labeller=lambda convo: convo.meta['thread_doubles'])

Initializing default classification model (standard scaled logistic regression)


In [30]:
bow_classifier2.fit_transform(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [31]:
summary = bow_classifier2.summarize(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [32]:
summary.head()

,prediction,pred_score
id,,
dt05qyf,True,1.0
dandio0,True,1.0
dwa6k96,True,1.0
dsldpxg,True,1.0
e70wjy3,True,1.0


In [33]:
summary.tail()

,prediction,pred_score
id,,
drduxx1,False,2.464949e-12
dl7q7n2,False,8.294859e-14
dxfib8r,False,2.744422e-15
dwqaa06,False,2.809290e-16
d8y9akn,False,1.628800e-16


In [34]:
bow_classifier2.base_accuracy(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

0.6761904761904762

In [35]:
bow_classifier2.accuracy(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

0.9992063492063492

In [36]:
print(bow_classifier2.classification_report(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00       852
        True       1.00      1.00      1.00       408

    accuracy                           1.00      1260
   macro avg       1.00      1.00      1.00      1260
weighted avg       1.00      1.00      1.00      1260



In [37]:
bow_classifier2.get_coefs(feature_names=bow_transformer2.get_vocabulary()).head(10)

,coef
feat_name,
nothing,0.611803
house,0.513433
common,0.508522
stop,0.481769
saw,0.463165
removed,0.456147
media,0.441399
profile,0.417387
500,0.414077


In [38]:
bow_classifier2.get_coefs(feature_names=bow_transformer2.get_vocabulary()).tail(10)

,coef
feat_name,
hill,-0.371316
added,-0.383548
tried,-0.383805
nice,-0.387194
70,-0.403627
god,-0.416933
joke,-0.419558
goes,-0.427241
enjoy,-0.491965


In [39]:
bow_classifier2.confusion_matrix(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

array([[852,   0],
       [  1, 407]])

In [40]:
import pandas as pd

In [41]:
bow_classifier2.evaluate_with_cv(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

Using corpus objects...

Running a cross-validated evaluation...
Done.


array([0.65079365, 0.56746032, 0.59126984, 0.58730159, 0.6031746 ])

In [42]:
bow_classifier2.evaluate_with_train_test_split(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

Using corpus objects...

Running a train-test-split evaluation...
Done.


(0.5793650793650794, array([[108,  51],
        [ 55,  38]]))

In [43]:
objs = list(threads_corpus.iter_conversations(selector=lambda convo: convo.meta['thread_doubles'] is not None))

In [44]:
objs = bow_classifier2.transform_objs(objs)

Using input list of corpus objects...



In [45]:
bow_classifier2.summarize_objs(objs)

,prediction,pred_score
id,,
e447bx1,False,8.868021e-13
e1quixk,False,3.485643e-12
e0iez9l,False,7.631712e-12
c3n0z5p,False,1.985637e-11
dl7q7n2,False,2.557649e-11
ctdyekm,False,2.918384e-11
dl64gc9,False,5.108080e-11
dwqaa06,False,8.338774e-11
d8y9akn,False,1.351222e-10


## Paired bag-of-words prediction for comment thread doubling in length versus staying the same length based on first 5 utterances

In [46]:
from convokit import Pairer, PairedBoW

In [47]:
corpus = Corpus(filename=download('subreddit-Cornell'))

Dataset already exists at /Users/calebchiam/.convokit/downloads/subreddit-Cornell


In [48]:
threads_corpus = corpus.reindex_conversations(new_convo_roots=top_level_comment_ids, preserve_convo_meta=True)


['c3od15i', 'c3oyf4d', 'c3p8bze', 'c3p1rn8', 'c3ocsyl']


In [49]:
next(threads_corpus.iter_conversations())

Conversation({'obj_type': 'conversation', '_owner': <convokit.model.corpus.Corpus object at 0x139106a50>, 'meta': {'original_convo_meta': {'title': "What are some things that I should buy for my dorm that Cornell doesn't tell me about?", 'num_comments': 9, 'domain': 'self.Cornell', 'timestamp': 1533313697, 'subreddit': 'Cornell', 'gilded': 0, 'gildings': None, 'stickied': False, 'author_flair_text': 'A&amp;S 2022'}, 'original_convo_id': '94b1uu'}, '_id': 'e3jmnsh', '_utterance_ids': ['e3jmnsh', 'e3jne5u', 'e3joeer', 'e3jrlny'], '_speaker_ids': None, 'tree': None})

In [50]:
for thread in threads_corpus.iter_conversations():
    thread_len = len(list(thread.iter_utterances()))
    if thread_len == 5:
        thread.meta['thread_doubles'] = False
    elif thread_len >= 10:
        thread.meta['thread_doubles'] = True
    else:
        thread.meta['thread_doubles'] = None

In [51]:
next(threads_corpus.iter_conversations()).meta

{'original_convo_meta': {'title': "What are some things that I should buy for my dorm that Cornell doesn't tell me about?",
  'num_comments': 9,
  'domain': 'self.Cornell',
  'timestamp': 1533313697,
  'subreddit': 'Cornell',
  'gilded': 0,
  'gildings': None,
  'stickied': False,
  'author_flair_text': 'A&amp;S 2022'},
 'original_convo_id': '94b1uu',
 'thread_doubles': None}

In [52]:
pairer = Pairer(obj_type="conversation", 
                pos_label_func=lambda convo: convo.meta['thread_doubles'], 
                neg_label_func=lambda convo: convo.meta['thread_doubles'] == False,
                pairing_func=lambda convo: convo.meta['original_convo_id']
               )

In [53]:
pairer.transform(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [54]:
print(next(threads_corpus.iter_conversations()))

Conversation('id': 'e3jmnsh', 'utterances': ['e3jmnsh', 'e3jne5u', 'e3joeer', 'e3jrlny'], 'meta': {'original_convo_meta': {'title': "What are some things that I should buy for my dorm that Cornell doesn't tell me about?", 'num_comments': 9, 'domain': 'self.Cornell', 'timestamp': 1533313697, 'subreddit': 'Cornell', 'gilded': 0, 'gildings': None, 'stickied': False, 'author_flair_text': 'A&amp;S 2022'}, 'original_convo_id': '94b1uu', 'thread_doubles': None, 'pair_obj_label': None, 'pair_id': None, 'pair_orientation': None})


In [55]:
for convo in threads_corpus.iter_conversations():
    if convo.meta['pair_id'] is not None:
        print(convo)
        break

Conversation('id': 'cwezqjo', 'utterances': ['cwezqjo', 'cwf0lpd', 'cwf2ay5', 'cwf2vif', 'cwf3v3k'], 'meta': {'original_convo_meta': {'title': "Watters' World Gets Shut Down at Cornell University", 'num_comments': 14, 'domain': 'insider.foxnews.com', 'timestamp': 1445958710, 'subreddit': 'Cornell', 'gilded': 0, 'gildings': None, 'stickied': False, 'author_flair_text': ''}, 'original_convo_id': '3qff0l', 'thread_doubles': False, 'pair_obj_label': 'neg', 'pair_id': '3qff0l', 'pair_orientation': 'neg'})


In [56]:
from convokit import BoWTransformer

In [57]:
bow_transformer = BoWTransformer(obj_type="conversation", 
                                 text_func=lambda convo: " ".join([utt.text for utt in convo.get_chronological_utterance_list()][:5]),
                    )

Initializing default unigram CountVectorizer...


In [58]:
bow_transformer.fit_transform(threads_corpus, selector=lambda convo: convo.meta['pair_id'] is not None)

In [59]:
paired_bow = PairedBoW(obj_type="conversation")

In [60]:
paired_bow.fit(threads_corpus)

Found 65 valid pairs.


In [61]:
paired_bow.summarize(threads_corpus)

Found 65 valid pairs.


0.4461538461538462

In [62]:
paired_bow.get_coefs(feature_names=bow_transformer.get_vocabulary())

,coef
feat_name,
accepted,0.362877
school,0.334063
shit,0.316190
similar,0.284726
ve,0.263209
computer,0.262590
since,0.259912
him,0.249187
speak,0.248708
